In [ ]:
using Markdown #hide

# C₄ᵥ CTMRG and QR-CTMRG

In this example we demonstrate specialized CTMRG variants that exploit the $C_{4v}$ point group
symmetry of the physical model and PEPS at hand. This allows us to significantly reduce the
computational cost of contraction and optimization. To that end, we will consider the Heisenberg
Hamiltonian on the square lattice

$$
H = \sum_{\langle i,j \rangle} \left ( J_x S^{x}_i S^{x}_j + J_y S^{y}_i S^{y}_j + J_z S^{z}_i S^{z}_j \right )
$$

We want to treat the model in the antiferromagnetic regime where the ground state exhibits
bipartite sublattice structure. To be able to represent this ground state in a $C_{4v}$-invariant
manner on a single-site unit cell, we perform a unitary sublattice rotation by setting
$(J_x, J_y, J_z)=(-1, 1, -1)$.

Let's get started by seeding the RNG and doing the imports:

In [ ]:
using Random
using TensorKit, PEPSKit
Random.seed!(123456789);

## Defining a specialized Hamiltonian for C₄ᵥ-symmetric PEPS

Since the model under consideration and its ground state are invariant under 90° rotation and
Hermitian reflection, evaluating the expectation values of the horizontal and vertical energy
contributions are exactly equivalent. This allows us to effectively halve the computational cost
by evaluating only half of the terms and multiplying by 2. In practice, we implement this
using a specialized `LocalOperator` that contains only the relevant terms:

In [ ]:
using MPSKitModels: S_xx, S_yy, S_zz

# Heisenberg model assuming C4v symmetric PEPS and environment, which only evaluates necessary term
function heisenberg_XYZ_c4v(lattice::InfiniteSquare; kwargs...)
    return heisenberg_XYZ_c4v(ComplexF64, Trivial, lattice; kwargs...)
end
function heisenberg_XYZ_c4v(
        T::Type{<:Number}, S::Type{<:Sector}, lattice::InfiniteSquare;
        Jx = -1.0, Jy = 1.0, Jz = -1.0, spin = 1 // 2,
    )
    @assert size(lattice) == (1, 1) "only trivial unit cells supported by C4v-symmetric Hamiltonians"
    term =
        rmul!(S_xx(T, S; spin = spin), Jx) +
        rmul!(S_yy(T, S; spin = spin), Jy) +
        rmul!(S_zz(T, S; spin = spin), Jz)
    spaces = fill(domain(term)[1], (1, 1))
    return LocalOperator( # horizontal and vertical contributions are identical
        spaces, (CartesianIndex(1, 1), CartesianIndex(1, 2)) => 2 * term
    )
end;

## Initializing C₄ᵥ-invariant PEPSs and environments

In order to use $C_{4v}$-symmetric algorithms, it is of course crucial to use initial guesses
with $C_{4v}$ symmetry. First, we create a real-valued random PEPS that we explicitly
symmetrize using `symmetrize!` and the $C_{4v}$ symmetry `RotateReflect`:

In [ ]:
symm = RotateReflect()
D = 2
T = Float64
peps_random = InfinitePEPS(randn, T, ComplexSpace(2), ComplexSpace(D))
peps₀ = symmetrize!(peps_random, symm);

Initializing an $C_{4v}$-invariant environment is a bit more subtle and there is no one-size-fits-all
solution. As a good starting point one can use the initialization function `initialize_random_c4v_env`
(or also `initialize_singlet_c4v_env`) where we construct a diagonal corner with random
real entries and a random Hermitian edge tensor.

In [ ]:
χ = 16
env_random_c4v = initialize_random_c4v_env(peps₀, ComplexSpace(χ));

Then contracting the PEPS using $C_{4v}$ CTMRG is as easy as just calling `leading_boundary`
but passing the initial PEPS and environment as well as the `alg = :c4v` keyword argument:

In [ ]:
env₀, = leading_boundary(env_random_c4v, peps₀; alg = :c4v, tol = 1.0e-10);

## C₄ᵥ-symmetric optimization

We now take `peps₀` and `env₀` as a starting point for a gradient-based energy
minimization where we contract using $C_{4v}$ CTMRG such that the energy gradient will also
exhibit $C_{4v}$ symmetry. For that, we call `fixedpoint` and specify `alg = :c4v`
as the boundary contraction algorithm:

In [ ]:
H = real(heisenberg_XYZ_c4v(InfiniteSquare())) # make Hamiltonian real-valued
peps, env, E, = fixedpoint(
    H, peps₀, env₀; optimizer_alg = (; tol = 1.0e-4), boundary_alg = (; alg = :c4v),
);

We note that this energy is slightly higher than the one obtained from an
optimization using asymmetric CTMRG with equivalent settings.
Indeed, this is what one would expect since the $C_{4v}$ symmetry restricts the PEPS ansatz
leading to fewer free parameters, i.e. an ansatz with reduced expressivity.
Comparing against Juraj Hasik's data from $J_1\text{-}J_2$
[PEPS simulations](https://github.com/jurajHasik/j1j2_ipeps_states/blob/main/single-site_pg-C4v-A1/j20.0/state_1s_A1_j20.0_D2_chi_opt48.dat),
we find very good agreement:

In [ ]:
E_ref = -0.6602310934799577 # Juraj's energy at D=2, χ=16 with C4v symmetry
@show (E - E_ref) / E_ref;

As a consistency check, we can compute the vertical and horizontal correlation lengths,
and should find that they are equal (up to the sparse eigensolver tolerance):

In [ ]:
ξ_h, ξ_v, = correlation_length(peps, env)
@show ξ_h ξ_v;

## QR-CTMRG

The conventional $C_{4v}$ CTMRG algorithm works by performing an eigendecomposition of
the enlarged corner $A = V D V^\dagger$, taking the diagonal eigenvalue tensor as the new
corner $C' = D$ and then renormalizing the edge using the isometry $V$. There exist modifications
to this standard algorithm, most notably *QR-CTMRG* as presented in a recent publication by
[Zhang, Yang and Corboz](@cite zhang_accelerating_2025). There the idea is to replace the
eigendecomposition by a QR decomposition of a lower-rank approximation of the enlarged corner.
While less accurate in some ways, the QR-CTMRG approach substantially accelerates contraction
and optimization times, and also has vastly improved GPU performance. Notably, it is found
that QR-CTMRG converges to the same fixed point as regular $C_{4v}$ CTMRG.

In PEPSKit terms, using QR-CTMRG just amounts to switching out the projector algorithm that is
used by the `C4vCTMRG` algorithm to `projector_alg = :c4v_qr` (as opposed to `:c4v_eigh`).
QR-CTMRG tends to need significantly more iterations to converge while still being much faster,
hence we need to increase `maxiter`:

In [ ]:
env_qr₀, = leading_boundary(
    env_random_c4v, peps; alg = :c4v, projector_alg = :c4v_qr, maxiter = 500,
);

To optimize using QR-CTMRG we proceed analogously by specifiying `projector_alg = :c4v_qr` and
increasing the `maxiter` when setting the boundary algorithm parameters. We make sure to supply
the `env_qr₀` initial environment because it does not use `DiagonalTensorMap`s as its corner
type (only regular `eigh`-based $C_{4v}$ CTMRG produces diagonal corners):

In [ ]:
peps_qr, env_qr, E_qr, = fixedpoint(
    H, peps₀, env_qr₀;
    optimizer_alg = (; tol = 1.0e-4),
    boundary_alg = (; alg = :c4v, projector_alg = :c4v_qr, maxiter = 500),
    gradient_alg = (; alg = :linsolver)
);
@show (E_qr - E_ref) / E_ref;

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*